In [3]:
from sqlalchemy import create_engine
import pandas as pd
import geopandas as gpd

engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
conn = engine.connect()

In [7]:
def read_file(script_path):
    """
    helper function to read sql script file and return sqlalchemy textual sql
    """
    from sqlalchemy.sql import text
    with open(script_path, 'r') as sql_file:
        sql_query = sql_file.read()

    return text(sql_query)

#### Daily, Weekly and Monthly average number of trips per region

In [12]:
sql = read_file('../scripts/avgs_by_region.sql')
df = pd.read_sql_query(sql,con=engine)
df

,region,daily_avg,weekly_avg,monthly_avg
0,Turin,1.727273,7.6,38.0
1,Hamburg,1.333333,5.6,28.0
2,Prague,1.619048,6.8,34.0


#### What regions has the "cheap_mobile" datasource appeared in?

In [13]:
sql = read_file('../scripts/cheap_mobile_counts.sql')
df = pd.read_sql_query(sql,con=engine)
df

,region,cheap_mobile_appearances
0,Prague,13
1,Hamburg,10
2,Turin,10


#### From the two most commonly appearing regions, which is the latest datasource?

In [14]:
sql = read_file('../scripts/latest_datasources.sql')
df = pd.read_sql_query(sql,con=engine)
df

,region,latest_datasource
0,Turin,pt_search_app
1,Prague,cheap_mobile
